# Module 4
## Step 3: Sensitivity Analysis

For this section, we will refer primarily to methods indicated in Ligmann-Zielinska et al. (2020) (LZ 2020), Kwakkel (2017), and course materials.

As we mention in the previous section, the Altruism model is a more "abstract" model. Therefore, it seems that an appropriate route for sensitivity analysis (SA) would be to start with a One at A Time (OAT) SA, as recommended by Figure 3 in LZ 2020. Note that here we have selected a simpler OAT rather than an One Factor At a Time--we are stepping through all of the parameters at regular intevals (see in the ranges below) rather than using random intervals. We do this with the intent to better understand the model and choose a next analysis.



In [1]:
import os
import sys
import numpy as np
import pandas as pd

# Add the parent directory to the Python path
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from utils.pynetlogo_utils import run_altruism_experiment, process_results


base_altruism_model = "./M4Model_Dresslar_base.nlogo"  # (copied into this directory!)

max_ticks = 15000  # so expensive. many runs collapse right away, but if they do not we need to know why.

# important: run_altruism_experiment needs arrays even if we have single value params, which we do.
# keep the first two params at 0.26
altruistic_probability = [0.26]
selfish_probability = [0.26]

# four ranges, one for each sensitivity param
cost_range = np.linspace(0, 1.0, 5) # note that > 0.9 is OOB
benefit_range = np.linspace(0, 1.0, 5) # note that > 0.9 is OOB
disease_range = np.linspace(0, 1.0, 5) # inclusive of 1.0
harshness_range = np.linspace(0, 1.0, 5) # inclusive of 1.0
runs_per_node = 10

As requested in the assignment, we are exploring the (altruism) cost, (altruism) benefit, disease, and harshness parameters of the model. Our currency is the populations of "altruists" and "selfish" at the completion of the model. We might note that a specific model behavior is the collapse and extinction of one of the populations at a certain point, and that the duration of resilience of one group or the other that becomes extinct might be a useful secondary measure: for instance, we might class a run in which the altruists reach zero population in 20 ticks to be different from a run in which they do so at tick 2000. 

So, for our OAT testing, weʻll attempt to explore that factor just to understand the parameter settings at which lengthy meaningful runs are possible. To do this, our model is modified to stop on the tick when either altruists of selfish reach zero population, and we set up a very large maximum ticks value in order to account for long (within practical reason) duration runs. At certain starting parameters, the model can stay quite dynamic, with both populations swapping positions of advantgae multiple times, even for thousands of ticks.

Our runs are set up again using `pynetlogo`, this time working with a separate module that contains the running code for convenience. The experimental harness code has changed very little from our previous notebook and the M2 module assignment, with the exception that it has been designed to be possible to stop and start from a checkpoint.

The `pynetlogo` code will sweep the designated parameters listed in the codeblock above. Note that `altuistic_probability` and `selfish_probability` are left at their defaults. 

As for `runs_per_node`, first it should be explained that we call a multi-paramater settings point on our 4x5 matrix (formed using the ranges) a "multi-parameter node" or "node." Since this is an exploration the decision has been made to simply always perform as many runs as practically possible given time constraints. 10 runs per "node" seem sufficient, and we will mark in our analysis parameter settings for which there were multiple "outcomes" within the nodes.

In [2]:
# conservatively run a first pass to see if we can find breakpoints. similar to our searching in M2.

# results_first_search_filename = run_altruism_experiment(base_altruism_model, 
                                            #  "first_search", 
                                            #  max_ticks,
                                            #  runs_per_node,
                                            #  altruistic_probability, 
                                            #  selfish_probability, 
                                            #  cost_range, 
                                            #  benefit_range,
                                            #  disease_range,
                                            #  harshness_range)


Once this has been completed, we have a CSV file with results for each run for each node. The header and first twelve rows look like:

In [3]:
df = pd.read_csv("1st_search_results.csv")
df.head(12)
# or, df.tail()

,altruistic_probability,selfish_probability,cost_of_altruism,benefit_from_altruism,disease,harshness,run_number,ticks,altruists,selfish,void,max_pink,max_green,max_black
0,0.26,0.26,0.0,0.0,0.0,0.00,0,1062.0,0.0,1681.0,0.0,825.0,1681.0,39.0
1,0.26,0.26,0.0,0.0,0.0,0.00,1,5631.0,1681.0,0.0,0.0,1681.0,1234.0,50.0
2,0.26,0.26,0.0,0.0,0.0,0.00,2,2669.0,1681.0,0.0,0.0,1681.0,876.0,33.0
3,0.26,0.26,0.0,0.0,0.0,0.00,3,800.0,1681.0,0.0,0.0,1681.0,901.0,45.0
4,0.26,0.26,0.0,0.0,0.0,0.00,4,3170.0,1681.0,0.0,0.0,1681.0,831.0,40.0
5,0.26,0.26,0.0,0.0,0.0,0.00,5,5225.0,0.0,1681.0,0.0,805.0,1681.0,42.0
6,0.26,0.26,0.0,0.0,0.0,0.00,6,2931.0,0.0,1681.0,0.0,1243.0,1681.0,44.0
7,0.26,0.26,0.0,0.0,0.0,0.00,7,15000.0,1124.0,557.0,0.0,1499.0,1626.0,37.0
8,0.26,0.26,0.0,0.0,0.0,0.00,8,2383.0,0.0,1681.0,0.0,935.0,1681.0,45.0
9,0.26,0.26,0.0,0.0,0.0,0.00,9,6573.0,0.0,1681.0,0.0,1052.0,1681.0,42.0


As we can see from our data, harshness is adjusted starting on data row 10. Ticks numbers are very high in this tiny sample of parameters, it might be noted; starting the simulation with "all zeros" does indeed lead to some very long and unpredictable runs. The vast majority of the parameter nodes sampled collapse far sooner, frequently well within 100 ticks. In order to use our data to perform analyses, it seems useful to summarize the runs into means and standard deviations, which we will do here:

In [4]:
results_first_search_filename = "first_search_results.csv"

first_search_stats_filename = process_results(results_first_search_filename, runs_per_node, max_ticks)

Interesting results for node 0: ['variable ticks_outcomes', 'variable altruists_died_outcomes', 'variable selfish_died_outcomes']
node params: cost: 0.0, benefit: 0.0, disease: 0.0, harshness: 0.0
Interesting results for node 1: ['variable ticks_outcomes', 'variable altruists_died_outcomes', 'variable selfish_died_outcomes']
node params: cost: 0.0, benefit: 0.0, disease: 0.0, harshness: 0.25
Interesting results for node 2: ['variable altruists_died_outcomes', 'variable selfish_died_outcomes']
node params: cost: 0.0, benefit: 0.0, disease: 0.0, harshness: 0.5
Interesting results for node 3: ['variable altruists_died_outcomes', 'variable selfish_died_outcomes']
node params: cost: 0.0, benefit: 0.0, disease: 0.0, harshness: 0.75
Interesting results for node 4: ['variable ticks_outcomes', 'variable altruists_died_outcomes', 'variable selfish_died_outcomes']
node params: cost: 0.0, benefit: 0.0, disease: 0.0, harshness: 1.0
Interesting results for node 5: ['variable altruists_died_outcomes'

We now have available a file with all our mean and standard deviations for each measured output (currency) for each 10-run bundle for each "node" in our OAT matrix. Additionally, we have asked our res

### Notes

### References

Kwakkel, J. H., & Jaxa-Rozen, M. (2017). Example 2: Sensitivity analysis for a NetLogo model with SALib and ipyparallel. pyNetLogo Documentation. https://pynetlogo.readthedocs.io/en/latest/docs/SALib_ipyparallel.html

Ligmann-Zielinska, A., Siebers, P. O., Magliocca, N., Parker, D. C., Grimm, V., Du, J., ... & Ye, X. (2020). ‘One size does not fit all’: A roadmap of purpose-driven mixed-method pathways for sensitivity analysis of agent-based models. Journal of Artificial Societies and Social Simulation, 23(1).